In [1]:
import sys
sys.path.append("../../../experiments/") # use utils

import utils
import importlib
importlib.reload(utils)

from utils import calc_auc, get_abstract, normalize_text

import requests
import json
from progressbar import progressbar as pb
from utils import API_KEY, SEARCH_ENGINE_ID
from copy import deepcopy

In [2]:
api_key = API_KEY
search_engine_id = SEARCH_ENGINE_ID

url = f"https://www.googleapis.com/customsearch/v1?key={api_key}&cx={search_engine_id}&q=<QUERY>"

In [3]:
df = pd.read_csv("../../../data/data_to_process.csv")
print(len(df))
df.head(3)

113


,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


## Keywords

### Pt. 1

In [9]:
for_dataframe = {item: [] for item in ["query_id", "data_source", "pubmed_id"]}

for _, row in pb(df.iterrows(), max_value=len(df)):
    query_id = row.query_id
    data_source = row.data_source
    query = row.query
    
    new_url = deepcopy(url)
    new_url = new_url.replace("<QUERY>", query)
    response = requests.get(new_url, headers={'Content-Type': 'application/json'}).json()
    
    if "items" in response:
        for resp in response['items']:
            doc_id = resp['link']
            for_dataframe['query_id'].append(query_id)
            for_dataframe['data_source'].append(data_source)
            for_dataframe['pubmed_id'].append(doc_id)

100% (113 of 113) |######################| Elapsed Time: 0:01:07 Time:  0:01:07


In [ ]:
fin_df = (
    pd.DataFrame(for_dataframe)
    .merge(df, on=["query_id", "data_source"])
)

In [12]:
fin_df.to_csv("google_search_wiki_keywords_part_1.csv", index=0)

### Pt. 2

In [ ]:
pt1 = pd.read_csv("google_search_wiki_keywords_part_1.csv")
pt2_to_search = (
    df
    .merge(pt1, how='left', indicator=True)
    .query("_merge == 'left_only'")
    .drop(["pubmed_id", '_merge'], axis=1)
    .reset_index(drop=True)
)

In [17]:
for_dataframe = {item: [] for item in ["query_id", "data_source", "pubmed_id"]}

for _, row in pb(pt2_to_search.iterrows(), max_value=len(pt2_to_search)):
    query_id = row.query_id
    data_source = row.data_source
    query = row.query
    
    new_url = deepcopy(url)
    new_url = new_url.replace("<QUERY>", query)
    response = requests.get(new_url, headers={'Content-Type': 'application/json'}).json()
    
    if "items" in response:
        for resp in response['items']:
            doc_id = resp['link']
            for_dataframe['query_id'].append(query_id)
            for_dataframe['data_source'].append(data_source)
            for_dataframe['pubmed_id'].append(doc_id)

100% (9 of 9) |##########################| Elapsed Time: 0:00:09 Time:  0:00:09


In [ ]:
fin_df = (
    pd.DataFrame(for_dataframe)
    .merge(pt2_to_search, on=["query_id", "data_source"])
)

In [19]:
fin_df.to_csv("google_search_wiki_keywords_part_2.csv", index=0)

## Question

### Pt. 1

In [21]:
df = pd.read_csv("../../../data/data_to_process.csv")
print(len(df))
df.head(3)

113


,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


In [24]:
for_dataframe = {item: [] for item in ["query_id", "data_source", "pubmed_id"]}

for _, row in pb(df.iterrows(), max_value=len(df)):
    query_id = row.query_id
    data_source = row.data_source
    query = normalize_text(row.description, False).strip()
    
    new_url = deepcopy(url)
    new_url = new_url.replace("<QUERY>", query)
    response = requests.get(new_url, headers={'Content-Type': 'application/json'}).json()
    
    if "items" in response:
        for resp in response['items']:
            doc_id = resp['link']
            for_dataframe['query_id'].append(query_id)
            for_dataframe['data_source'].append(data_source)
            for_dataframe['pubmed_id'].append(doc_id)

100% (113 of 113) |######################| Elapsed Time: 0:01:36 Time:  0:01:36


In [25]:
fin_df = (
    pd.DataFrame(for_dataframe)
    .merge(df, on=["query_id", "data_source"])
)
print(len(fin_df))
fin_df.head(2)

945


,query_id,data_source,pubmed_id,description,query,label
0,1,2019,https://en.wikipedia.org/wiki/Urinary_tract_in...,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,1,2019,https://en.wikipedia.org/wiki/Proanthocyanidin,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0


In [26]:
fin_df.to_csv("google_search_wiki_question_part_1.csv", index=0)

### Pt. 2

In [31]:
df = pd.read_csv("../../../data/data_to_process.csv")
df.head(3)

,data_source,query_id,description,query,label
0,2019,1,Can cranberries prevent urinary tract infections?,cranberries urinary tract infections,0.0
1,2019,3,Can acupuncture be effective for people with e...,acupuncture epilepsy,0.0
2,2019,5,Can acupuncture prevent migraines?,acupuncture migraine,1.0


In [ ]:
pt1 = pd.read_csv("google_search_wiki_question_part_1.csv")
pt1 = pt1.astype({"query_id": object})
pt2_to_search = (
    df
    .merge(pt1, on=['query', 'description', 'data_source', 'label'], how='left', indicator=True)
    .query("_merge == 'left_only'")
    .drop(["pubmed_id", '_merge', 'query_id_y'], axis=1)
    .rename(columns={"query_id_x": "query_id"})
    .reset_index(drop=True)
)

In [41]:
for_dataframe = {item: [] for item in ["query_id", "data_source", "pubmed_id"]}

for _, row in pb(pt2_to_search.iterrows(), max_value=len(pt2_to_search)):
    query_id = row.query_id
    data_source = row.data_source
    query = row.query
    
    new_url = deepcopy(url)
    new_url = new_url.replace("<QUERY>", query)
    response = requests.get(new_url, headers={'Content-Type': 'application/json'}).json()
    
    if "items" in response:
        for resp in response['items']:
            doc_id = resp['link']
            for_dataframe['query_id'].append(query_id)
            for_dataframe['data_source'].append(data_source)
            for_dataframe['pubmed_id'].append(doc_id)

100% (17 of 17) |########################| Elapsed Time: 0:00:14 Time:  0:00:14


In [ ]:
fin_df = (
    pd.DataFrame(for_dataframe)
    .merge(pt2_to_search, on=["query_id", "data_source"])
)

In [46]:
fin_df.to_csv("google_search_wiki_question_part_2.csv", index=0)

## Process retrieved wiki articles

In [ ]:
import pandas as pd
import wikipedia
from wikipedia.exceptions import DisambiguationError, PageError

import warnings
warnings.filterwarnings("ignore")

from utils import normalize_text

In [ ]:
init_data = pd.read_csv("../../../data/data_to_process.csv")
init_data.head(3)

In [ ]:
def process_wiki_url(url):
    last = url.split("/")[-1]
    if "Talk%" in url or url.endswith(".jpg") or \
    ("en.wikipedia.org" not in url and "simple.wikipedia.org" not in url) \
    or "Wikipedia:Reference_desk" in url:
        return ""
    
    last = re.sub("\%[A-Z0-9]{2}", " ", last)
    last = re.sub(" +", " ", last)
    
    return last.replace("_", " ").strip().strip("-")

In [ ]:
TOP_K_PER_QUERY = 10 # maximum number of paragraphs that we are looking for one query
TOP_K_PER_ARTICLE = 2 # maximum number of paragraphs we take from one wiki article

def get_suitable_paragraphs(query, tokens_to_find, content, serp_pos, title, title_common_tokens, top_k_per_article=TOP_K_PER_ARTICLE):
    
    paragraphs = content.split("\n")
    
    result = {key: [] for key in [
        "query_processed",
        "article_title", "paragraph", "serp_position", 
        "title_common_tokens", "num_title_common_tokens", 
        "tokens_to_find", 
        "paragraph_common_tokens", "num_paragraph_common_tokens",
        "paragraph_number_in_article"
    ]}
    
    for i, p in enumerate(paragraphs):
        
        if not p or p.endswith("=="):
            continue
        
        normed_p = normalize_text(p)
        p_tokens = set(normed_p.split())
        
        paragraph_common_tokens = tokens_to_find & p_tokens
        
        if paragraph_common_tokens:
            result["query_processed"].append(query)
            result["article_title"].append(title)
            result["paragraph"].append(p)
            result["serp_position"].append(serp_pos)
            result["title_common_tokens"].append(list(title_common_tokens))
            result["num_title_common_tokens"].append(len(title_common_tokens))
            result["tokens_to_find"].append(list(tokens_to_find))
            result["paragraph_common_tokens"].append(list(paragraph_common_tokens))
            result["num_paragraph_common_tokens"].append(len(paragraph_common_tokens))
            result["paragraph_number_in_article"].append(i)

    return result


def get_paragraphs(query, wiki_article_names, top_k_per_query=TOP_K_PER_QUERY, top_k_per_article=TOP_K_PER_ARTICLE):
    
    normed_query = normalize_text(query)
    query_tokens = normed_query.split()
    
    fin_df = pd.DataFrame()
    
    for serp_pos, article_name in enumerate(wiki_article_names):
        try:
            wiki_page = wikipedia.page(article_name, auto_suggest=False)
        except DisambiguationError as e:
            print("Disambiguation error:", article_name)
        except PageError as e:
            print("PageError error:", article_name)
            continue
            
        title = wiki_page.title
        content = wiki_page.content
        
        normed_title = normalize_text(title)
        title_tokens = normed_title.split()

        tokens_to_find = set(query_tokens) - set(title_tokens)
        title_common_tokens = set(query_tokens) & set(title_tokens)

        if len(title_common_tokens) == 0: # ignore if no query tokens are presented in title
            continue
        
        result = get_suitable_paragraphs(query, tokens_to_find, content, serp_pos + 1, title, title_common_tokens)       

        result_df = (
            pd.DataFrame(result)
            .sort_values(
                by=["serp_position", "num_title_common_tokens", "num_paragraph_common_tokens", "paragraph_number_in_article"],
                ascending=[True, False, False, True]
            )
            .head(top_k_per_article)
        )
        fin_df = pd.concat((fin_df, result_df))
        
    if len(fin_df):
        fin_df = (
                pd.DataFrame(fin_df)
                .sort_values(
                    by=["serp_position", "num_title_common_tokens", "num_paragraph_common_tokens", "paragraph_number_in_article"],
                    ascending=[True, False, False, True]
                )
                .head(top_k_per_query)
            )

    return fin_df

## Keywords

In [ ]:
df = pd.concat((
    pd.read_csv("google_search_wiki_keywords_part_1.csv"),
    pd.read_csv("google_search_wiki_keywords_part_2.csv")
)).reset_index(drop=True)

In [ ]:
df["processed_url"] = df["pubmed_id"].apply(process_wiki_url)
df = df.query("processed_url != ''").reset_index(drop=True)

In [ ]:
final_result_query = pd.DataFrame()

for _, group in pb(df.groupby(["query_id", "data_source"]), max_value=len(init_data)):
    q = group["query"].tolist()[0]
    articles = group.processed_url.tolist()
    res_df = get_paragraphs(q, articles)
    
    if len(res_df):
        res_df['query_id'] = group.query_id.tolist()[0]
        res_df['data_source'] = group.data_source.tolist()[0]
        res_df['label'] = group.label.tolist()[0]

        final_result_query = pd.concat((final_result_query, res_df))

In [ ]:
final_result_query.to_csv("../../../data/google_search_wiki/wikipedia_articles_keywords.csv", index=False)

## Question

In [ ]:
df = pd.concat((
    pd.read_csv("google_search_wiki_question_part_1.csv"),
    pd.read_csv("google_search_wiki_question_part_2.csv")
)).reset_index(drop=True)

In [ ]:
final_result_query = pd.DataFrame()

for _, group in pb(df.groupby(["query_id", "data_source"]), max_value=len(init_data)):
    q = group["description"].tolist()[0]
    articles = group.processed_url.tolist()
    res_df = get_paragraphs(q, articles)
    
    if len(res_df):
        res_df['query_id'] = group.query_id.tolist()[0]
        res_df['data_source'] = group.data_source.tolist()[0]
        res_df['label'] = group.label.tolist()[0]

        final_result_query = pd.concat((final_result_query, res_df))

In [ ]:
final_result_query.to_csv("../../../data/google_search_wiki/wikipedia_articles_question.csv", index=False)